In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from functools import reduce

ModuleNotFoundError: No module named 'seaborn'

In [4]:
def read_cred():
    f = open("../../../cred.txt", "r")
    cred = f.read()
    f.close()
    return cred

In [5]:
read_cred()

'UU0xV1IyMU5NQUhERlBLVk01Mjo2NDRtaVREUjFjdDRGemc2RXJMTlF4VFZnZm9PbFBrSTh0bWF1WVFF'

In [6]:
def get_OI(url,start,end,interval='PT1M',tag='xx',auth='xx',hS='00',hF='23'):
	url_all =url+'data-reference='+tag+'&aggregation=TIME'+'&aggregation-function=MEAN'+"&from="+start+"T"+hS+"%3A00%3A00.000Z&to="+end+"T"+hF+"%3A59%3A59.000Z&aggregation-period="+interval
	# print(url_all,auth)
	d_data = pd.read_json(url_all,storage_options={ 'Authorization': 'basic '+ auth})
	# print(d_data['values'][0])
	arr = np.asarray(np.asarray(d_data['values'])[0])
	return d_data['values'][0]

In [7]:
urlBase = 'https://oianalytics-100.optimistik.fr/api/oianalytics/time-values/query?'
credentials = read_cred()

In [8]:
def get_data(tags):
    liste = list(range(0))
    for tag in tags:
        data = get_OI(urlBase,'2022-01-01','2022-06-06','PT20M',tag,credentials,'00','23')
        df = pd.DataFrame(data)
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        df = df.set_index('timestamp')
        df = df.rename(columns={'value':tag})
        liste.append(df)
    return liste

In [ ]:
def merge_data(liste):
    df = reduce(lambda left,right : pd.merge(left, right,left_index=True,right_index=True,how='outer'),liste)
    return df

In [ ]:
# tags = ['3B-T99994']
tags = ['E-T15223','E-T18223','3B-T99994','E-FC16325','E-FCS14325','E-FCS12085','E-FS31053','E-FC33005','E-FQ41504','E-QUR-E2']
#tags = ['E-T15223','E-T18223','E-FCS14325','E-FC16325','E-FCS12085','E-LS18603','E-LC18604','E-LS15603','E-LC15604','E-FS31053','E-PD15010','E-PD18010','3B-T99994','E-QUR-E2','ROC_E2_Prod_MMP_brut','E-FQ41504','E-TC18202','E-TC15202']
df_list = get_data(tags)
data = merge_data(df_list)
result = data.copy()
result.describe()

## Filtre

In [ ]:
def filtering(df):
    df = df[(df['E-QUR-E2']>500) & (df['E-QUR-E2']<600)]
    df = df[df['E-T15223']>390]
    df = df[df['E-FC16325']>8400]
    df = df[df['E-FCS12085']>48000]
    return df

In [ ]:
result = filtering(result)
result.describe()

In [ ]:
for tag in tags:
    plt.figure(figsize=(18,6))
# plt.plot(result[tags[0]],label=tags[0])
    plt.subplot(121)
    plt.scatter(result.index ,result[tag],label=tag, c='b', alpha=0.3)
    plt.legend()
    plt.subplot(122)
    sns.histplot(result[tag],label=tag,kde=True,common_norm=True)
    plt.legend()

In [ ]:
def get_perf(row):
    crit = 'E-QUR-E2'
    if row['E-QUR-E2'] > 540:
        ret=10
    elif row['E-QUR-E2']>535:
        ret=9
    elif row['E-QUR-E2']>530:
        ret=8
    elif row['E-QUR-E2']>525:
        ret=7
    elif row['E-QUR-E2']>520:
        ret=6
    elif row['E-QUR-E2']>515:
        ret=5
    elif row['E-QUR-E2']>510:
        ret=4
    elif row['E-QUR-E2']>505:
        ret=3
    elif row['E-QUR-E2']>500:
        ret=2
    elif row['E-QUR-E2']>495:
        ret=1
    else:
        ret =0
    return ret   

In [ ]:
result['perf'] = result.apply(get_perf,axis=1)
result = result.drop('E-QUR-E2',axis=1)
result.describe()

In [ ]:
freq = (result.perf.value_counts() / result.shape[0])
df = pd.DataFrame(freq).sort_index()
sns.barplot(data=df, x=df.index,y='perf',palette='RdBu')


In [ ]:
sns.color_palette("tab10")
sns.color_palette("rocket", as_cmap=True)
couleur = ['b','g','r','y','g']
sns.histplot(result['E-FC16325'],kde=True , color='g')

In [ ]:
sns.color_palette("tab10")
sns.heatmap(result.corr(),annot=True, cmap='RdBu')

In [ ]:
# sns.set_theme(style="ticks")
sns.set_theme(style='white',palette='Blues')
sns.color_palette("tab10")
sns.pairplot(data=result, hue='perf',palette='RdBu') 

# Machine Learning

### On sépare Trainset et Testset
et on vérifie que les deux jeux de données sont bien équilibrés

In [ ]:
from sklearn.model_selection import train_test_split
trainset, testset = train_test_split(result,test_size=0.8,random_state=0)
plt.figure(figsize=(10,7))
plt.subplot(121)
trainset['perf'].value_counts().plot.pie(autopct='%.0f%%')
plt.subplot(122)
testset['perf'].value_counts().plot.pie(autopct='%.0f%%')

## Preprocessing

In [ ]:
def encodage(df):
    return df

In [ ]:
def imputation(df):
    df =  df.dropna(axis=0)
    return df

In [ ]:
def feature_engineering(df):
    return df

In [ ]:
def preprocessing(df):
    df = encodage(df)
    df = feature_engineering(df)
    df = imputation(df)

    X = df.drop('perf',axis=1)
    y = df['perf']

    print(y.value_counts())
    return X,y 

In [ ]:
X_train, y_train = preprocessing(trainset)
X_train

In [ ]:
y_train.value_counts().plot.pie()

In [ ]:
X_test, y_test = preprocessing(trainset)

## Model de base et procédure d'évaluation

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.feature_selection import SelectKBest,f_classif
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA

# model = make_pipeline( PolynomialFeatures(2),SelectKBest(f_classif,k=10), RandomForestClassifier(random_state=0))

# model = make_pipeline( SelectKBest(f_classif,k=4), RandomForestClassifier(random_state=0))
model_1 = DecisionTreeClassifier(random_state=0)

In [ ]:
from sklearn.metrics import f1_score, confusion_matrix, classification_report
from sklearn.model_selection import learning_curve

In [ ]:
def evaluation(model):
    model.fit(X_train,y_train)
    y_pred = model.predict(X_test)

    print(confusion_matrix(y_test,y_pred))
    print(classification_report(y_test,y_pred))

    N, train_score, val_score = learning_curve(model,X_train,y_train,cv=4,train_sizes=np.linspace(0.1,1,10),scoring='f1')
    plt.figure(figsize=(12,8))
    plt.plot(N,train_score.mean(axis=1),c='b')
    plt.plot(N,val_score.mean(axis=1),c='y')

In [ ]:
evaluation(model_1)